# Clase 5: Aprendizaje no supervisado

A diferencia del aprendizaje supervisado, en el *análisis no supervisado* los datos no tienen etiqueta, y solo contamos con sus características $x_i$. 
Todos lucen igual y el objetivo es encontrar estructuras subyacentes en los datos: patrones, grupos, redundancias...:

<center>
<img src='im/NoSupervisado.PNG'> 
<img src='im/NoSupervisadoClas.PNG'>
</center>

Intentaremos dar respuesta a las siguientes preguntas:
- ¿Hay alguna manera informativa de visualizar los datos?
- ¿Existen subgrupos interesantes dentro de la muestra observada?
- ¿Hay datos atípicos o anomalías?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### ¿Qué tipos de datos es cada una de las variables?

In [ ]:
columnas = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Yearly Amount Spent']

In [ ]:
import seaborn as sns

## Reducción de dimensionalidad: Análisis de Componentes Principales (PCA)

- Queremos resumir la información según las direcciones de las características en las que más varían las observaciones.

- Un _Componente Principal_ $Z$ es una combinación lineal de las características de la muestra:

$$Z = \varphi_1 X_1 + \varphi_2 X_2 + ... + \varphi_k X_k.$$

Para obtener el _j_-ésimo componente principal $Z_j$ se procede de la siguiente manera manera:

$$Z_j = \varphi_{1,j} X_1 + \varphi_{2,j} X_2 + ... + \varphi_{k,j} X_k$$

$$\max_{\vec \varphi_j} \left\{ Var(Z_j) \right\} = \max_{\vec \varphi_j} \left\{ \frac{1}{N}\sum_{i=1}^{N}(\varphi_{1,j} x_{i,1} + \varphi_{2,j} x_{i,2} + ... + \varphi_{k,j} x_{k,N})^2 \right\},$$

con las restricciones 
$$\sum_{i=1}^{k}\varphi_{i,j}^2 = 1, ~~~~~ \vec\varphi_j \perp Z_1, ~\vec\varphi_j \perp Z_2, ..., ~\vec\varphi_j \perp Z_{j-1}.$$


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaled_data = ecommerce_data.loc[:, columnas]

In [ ]:
from sklearn.decomposition import PCA

Los componentes principales maximizan la varianza observada de los datos y, además, generan el hiperplano más cercano a las observaciones

<center>
<img src='im/PCAajuste.PNG'> 
</center>

Esto motiva a usarlos tambien como variables predictoras en modelos de aprendizaje supervisado.

### ¿Cuántos componentes usamos?

Buscamos un codo en la gráfica: cuando la ganancia en varianza explicada no compense el usar un componente adicional (más dimensionalidad).

### ¿Qué ocurre si no normalizamos las variables?

La diferencia en la escala genera que la variable de _gasto_ recoja el 99 % de la varianza y por tanto sea el primer componente principal exclusivamente.

#### Existen otros algoritmos de reducción de dimensionalidad:

- MCA.
- tSNE.

# Análisis de clustering: $k$-medias

Queremos divdir la base de datos en diferentes grupos tales que las observaciones en un mismo grupo sean _similares_ entre sí, y observaciones en grupos distintos sean _diferentes_ entre ellas.

**¿Cómo definimos dos datos como similares o diferentes?**


- El método de agrupamiento de $k$-medias busca dividir el conjunto de datos en $k$ grupos diferentes que no se sobrelapan.
- Cada una de las observaciones $x_i$ pertenece a uno, y solo uno, de los clusters $C_k$.
- Para cada grupo, se calcula su varianza interna (_within cluster variation_):

$$Var(C_k) = \sum_{j=1}^n Var(X_j^k) = \sum_{j=1}^n\left(\sum_{x_i\in C_k}(x_{i,j}-\bar X_j^k)^2\right).$$

- Queremos que la sumas de esta varianzas de los grupos sea lo más pequeña posible, en otras palabras, que los datos dentro de cada cluster sean lo más parecidos. 

In [ ]:
from sklearn.cluster import KMeans

### Los centroides de cada grupo nos permiten entender qué tipo de observaciones hay un cada uno de ellos.

### ¿Cuántos grupos se deben escoger?

- Información propia del problema.
- Nuevamente usamos la regla del codo: cuando la ganancia en menor varianza de los grupos no compense tener un grupo adicional.

### Existen muchos otros algoritmos de agrupamiento:

- $k$-prototipos.
- Clústering jerárquico.

# Detección de anomalías: Mixturas gaussianas

La idea es construir un _Modelo_ que permita estimar $p(x)$: la probabilidad de observar las características de cada uno de los datos. 

Con este modelo, si la probabilidad de observar $x_{test}$ es menor que cierto umbral de _rareza_: $p(x_{test}) < \varepsilon$, se cataloga $x_{test}$ como una observación _anómala_. Por el contrario, si $p(x_{test}) \geq \varepsilon$, $x_{test}$ supera el umbral de _rareza_ y se cataloga como una observación normal.

El Modelo de $p(x)$ debería indicar que observaciónes en el centro ocurren con mucha frecuencia,observaciones más alejadas ocurrirán frecuentemente, observaciones aún más alejadas ocurriren con relativa frecuencia, pero observaciones muy alejadas ocurrirán rara vez:

<center>
<img src='im/model_anom.PNG'> 
</center>

Una forma natural es suponer que las distintas variables se distribuyen de manera Normal:

$$p(x_i) = \prod_{j=1}^k \frac{1}{\sqrt{2\pi}\sigma_j}*\exp(-\frac{(x_{i,j}-\mu_j)}{2\sigma_j^2}),$$

con 

$$\mu_j = \frac{1}{N}\sum_{i=1}^N x_{i,j}, ~~~~ \sigma^2_j = \frac{1}{N} \sum_{i=1}^N (x_{i,j}-\mu_j)^2.$$


In [ ]:
test_data = ecommerce_data.loc[:, ['Time on Website', 'Yearly Amount Spent']]

In [ ]:
plt.scatter(test_data['Time on Website'], test_data['Yearly Amount Spent']);

In [ ]:
test_data['prob'] = ((1/((2*np.pi)**(1/2)*test_data['Time on Website'].std()))*np.exp(-(test_data['Time on Website']-test_data['Time on Website'].mean())**2/(2*test_data['Time on Website'].var()))*
                     (1/((2*np.pi)**(1/2)*test_data['Yearly Amount Spent'].std()))*np.exp(-(test_data['Yearly Amount Spent']-test_data['Yearly Amount Spent'].mean())**2/(2*test_data['Yearly Amount Spent'].var())))

In [ ]:
umbral_rareza = 0.00005

### Ahora usando _sklearn_

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gmm = GaussianMixture(covariance_type='full')

In [ ]:
test_data2 = ecommerce_data.loc[:, ['Time on Website', 'Yearly Amount Spent']]

In [ ]:
umbral_rareza = 2.3

#### Se puden eliminar algunos supuestos:

- Otras distribuciones de probabilidad.
- No independencia de las variables.

#### O usar otras metodologías:

- Isolation Forest.
- One Class SVM.